# Sutton & Barto Book

## Chapter 4: Dynamic Programming

## Demonstration of iterative policy evaluation

<IMG SRC="images/gridworld.png">


### Exercise 4.1

In Example 4.1, if $\pi$ is the equiprobable random policy,

- What is $q_\pi(11,down)$?
- What is $q_\pi(7,down)$?

In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
grid_shape = (4, 4)
terminal_state = [(0,0), (3, 3)]

states = []
for row in range(grid_shape[0]):
    for col in range(grid_shape[1]):
        p = (row, col)
        if not p in terminal_state[1]:
            states.append(p)
print(states)

[(0, 0), (0, 1), (0, 2), (0, 3), (1, 0), (1, 1), (1, 2), (1, 3), (2, 0), (2, 1), (2, 2), (2, 3), (3, 0), (3, 1), (3, 2), (3, 3)]


In [3]:
# State value-function
values = {s: 0 for s in states}
values

{(0, 0): 0,
 (0, 1): 0,
 (0, 2): 0,
 (0, 3): 0,
 (1, 0): 0,
 (1, 1): 0,
 (1, 2): 0,
 (1, 3): 0,
 (2, 0): 0,
 (2, 1): 0,
 (2, 2): 0,
 (2, 3): 0,
 (3, 0): 0,
 (3, 1): 0,
 (3, 2): 0,
 (3, 3): 0}

In [4]:
actions = {
            'l': (0, -1),
            'u': (-1, 0),
            'r': (0, 1),
            'd': (1, 0)
        }
actions

{'l': (0, -1), 'u': (-1, 0), 'r': (0, 1), 'd': (1, 0)}

In [5]:
def show_values(values, r=1):
    """Displays the values rounded to r decimal places.
    """
    
    v_array = np.zeros(grid_shape)
    for s, v in values.items():
        v_array[s] = v
    print(v_array.round(r))

show_values(values)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [6]:
# Q (state, action) value function
q_values = {}
for s in states:
    for a in actions:
        q_values[(s, a)] = 0.0
list(q_values.items())[0:3]

[(((0, 0), 'l'), 0.0), (((0, 0), 'u'), 0.0), (((0, 0), 'r'), 0.0)]

In [7]:
def move(s, a):
    """Returns next state and reward given current
    state (s) and action (a)."""

    if s in terminal_state:
        r = 0
    else:
        s2 = tuple(np.array(s) + np.array(actions[a]))

        # Check if out of bounds
        if (s2[0] < 0) or (s2[0] >= grid_shape[0]) or \
            (s2[1] < 0) or (s2[1] >= grid_shape[1]):
            s2 = s
        else:
            s = s2
        r = -1

    if s in terminal_state:
        s = terminal_state[0]

    return s, r

move((0, 1), 'r')

((0, 2), -1)

In [8]:
# Store all possible game transitions

transitions = {}
for s in states:
    for a in actions:
        transitions[(s, a)] = move(s, a)

list(transitions.items())

[(((0, 0), 'l'), ((0, 0), 0)),
 (((0, 0), 'u'), ((0, 0), 0)),
 (((0, 0), 'r'), ((0, 0), 0)),
 (((0, 0), 'd'), ((0, 0), 0)),
 (((0, 1), 'l'), ((0, 0), -1)),
 (((0, 1), 'u'), ((0, 1), -1)),
 (((0, 1), 'r'), ((0, 2), -1)),
 (((0, 1), 'd'), ((1, 1), -1)),
 (((0, 2), 'l'), ((0, 1), -1)),
 (((0, 2), 'u'), ((0, 2), -1)),
 (((0, 2), 'r'), ((0, 3), -1)),
 (((0, 2), 'd'), ((1, 2), -1)),
 (((0, 3), 'l'), ((0, 2), -1)),
 (((0, 3), 'u'), ((0, 3), -1)),
 (((0, 3), 'r'), ((0, 3), -1)),
 (((0, 3), 'd'), ((1, 3), -1)),
 (((1, 0), 'l'), ((1, 0), -1)),
 (((1, 0), 'u'), ((0, 0), -1)),
 (((1, 0), 'r'), ((1, 1), -1)),
 (((1, 0), 'd'), ((2, 0), -1)),
 (((1, 1), 'l'), ((1, 0), -1)),
 (((1, 1), 'u'), ((0, 1), -1)),
 (((1, 1), 'r'), ((1, 2), -1)),
 (((1, 1), 'd'), ((2, 1), -1)),
 (((1, 2), 'l'), ((1, 1), -1)),
 (((1, 2), 'u'), ((0, 2), -1)),
 (((1, 2), 'r'), ((1, 3), -1)),
 (((1, 2), 'd'), ((2, 2), -1)),
 (((1, 3), 'l'), ((1, 2), -1)),
 (((1, 3), 'u'), ((0, 3), -1)),
 (((1, 3), 'r'), ((1, 3), -1)),
 (((1, 3), '

In [9]:
def greedy_policy_with_values(state, action, values, transitions):
    
    # Requires the state-transition matrix
    action_values = [values[transitions[(state, a)][0]] for a in actions]
    v_max = max(action_values)
    a_greedy = list(actions.keys())[action_values.index(v_max)]

    return 1 if action == a_greedy else 0

In [10]:
def greedy_policy_with_q_values(state, action, q_values, transitions=None):

    action_values = [values[q_values[(state, a)]] for a in actions]
    v_max = max(action_values)
    a_greedy = list(actions.keys())[action_values.index(v_max)]

    return 1 if action == a_greedy else 0

In [11]:
def random_policy(state, action, values=None, transitions=None):

    return 1.0/len(actions)

In [12]:
def policy_action(policy, state, values, transitions=None):
    
    probs = [policy(state, a, values, transitions) for a in actions]
    
    return np.random.choice(list(actions.keys()), p=probs)

In [13]:
def bellman_equation(policy, states, actions, transitions,  
                     values, lr):
    """Updates the value function using the Bellman
    equation (4.5)."""
    
    v = values.copy()
    for s in states:
        
        sum_values = 0
        for a in actions:
            
            s2, r = transitions[(s, a)]
            v2 = values[s2]
            p = policy(s, a, values)
            sum_values += p*(r + lr*v2)

        v[s] = sum_values
    
    return v

In [14]:
def evaluate_policy(policy, states, actions, transitions,  
                    values, lr=1.0, theta=0.01, max_iter=1000,
                    show=True):

    iteration = 0
    if show:
            print("\nk =", iteration)
            show_values(values)

    while iteration < max_iter:

        updated_values = bellman_equation(policy, states, actions, 
                                          transitions, values, lr=lr)

        delta = np.abs(
            np.array(list(updated_values.values())) -
            np.array(list(values.values()))
        ).max()
        
        values = updated_values
        iteration += 1
        
        if show:
            print("\nk =", iteration)
            show_values(values)

        if delta < theta:
            break
    
    if iteration == max_iter:
        print("\nMaximum iterations reached.")
    else:
        print("\nConverged to delta < %f after %d iterations" % 
              (theta, iteration))
    
    return values

### 1. Equi-probable random policy

In [15]:
values = {s: 0 for s in states}

In [16]:
values = evaluate_policy(random_policy, states, actions, transitions, 
                         values, max_iter=10, show=True)


k = 0
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

k = 1
[[ 0. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1.  0.]]

k = 2
[[ 0.  -1.8 -2.  -2. ]
 [-1.8 -2.  -2.  -2. ]
 [-2.  -2.  -2.  -1.8]
 [-2.  -2.  -1.8  0. ]]

k = 3
[[ 0.  -2.4 -2.9 -3. ]
 [-2.4 -2.9 -3.  -2.9]
 [-2.9 -3.  -2.9 -2.4]
 [-3.  -2.9 -2.4  0. ]]

k = 4
[[ 0.  -3.1 -3.8 -4. ]
 [-3.1 -3.7 -3.9 -3.8]
 [-3.8 -3.9 -3.7 -3.1]
 [-4.  -3.8 -3.1  0. ]]

k = 5
[[ 0.  -3.7 -4.7 -4.9]
 [-3.7 -4.5 -4.8 -4.7]
 [-4.7 -4.8 -4.5 -3.7]
 [-4.9 -4.7 -3.7  0. ]]

k = 6
[[ 0.  -4.2 -5.5 -5.8]
 [-4.2 -5.2 -5.6 -5.5]
 [-5.5 -5.6 -5.2 -4.2]
 [-5.8 -5.5 -4.2  0. ]]

k = 7
[[ 0.  -4.7 -6.3 -6.7]
 [-4.7 -5.9 -6.4 -6.3]
 [-6.3 -6.4 -5.9 -4.7]
 [-6.7 -6.3 -4.7  0. ]]

k = 8
[[ 0.  -5.2 -7.  -7.5]
 [-5.2 -6.5 -7.1 -7. ]
 [-7.  -7.1 -6.5 -5.2]
 [-7.5 -7.  -5.2  0. ]]

k = 9
[[ 0.  -5.7 -7.7 -8.2]
 [-5.7 -7.2 -7.8 -7.7]
 [-7.7 -7.8 -7.2 -5.7]
 [-8.2 -7.7 -5.7  0. ]]

k = 10
[[ 0.  -6.1 -8.4 -9. ]
 [-6.1 -7.7 -8.4 

### 2. Greedy policy

In [17]:
values

{(0, 0): 0.0,
 (0, 1): -6.137969970703125,
 (0, 2): -8.35235595703125,
 (0, 3): -8.967315673828125,
 (1, 0): -6.137969970703125,
 (1, 1): -7.737396240234375,
 (1, 2): -8.427825927734375,
 (1, 3): -8.35235595703125,
 (2, 0): -8.35235595703125,
 (2, 1): -8.427825927734375,
 (2, 2): -7.737396240234375,
 (2, 3): -6.137969970703125,
 (3, 0): -8.967315673828125,
 (3, 1): -8.35235595703125,
 (3, 2): -6.137969970703125,
 (3, 3): 0.0}

In [18]:
policy_action(greedy_policy_with_values, (2,2), values, transitions)

'r'

In [19]:
def show_actions(states, policy, values, transitions=None):
    
    a_array = np.array(['_']*np.prod(grid_shape), dtype='<U1').reshape(grid_shape)
    for s in states:
        a_array[s] = policy_action(policy, s, values, transitions)
    
    print(a_array)

# Optimal actions
show_actions(states, greedy_policy_with_values, values, transitions)

[['l' 'l' 'l' 'l']
 ['u' 'l' 'l' 'd']
 ['u' 'u' 'r' 'd']
 ['u' 'r' 'r' 'l']]


### 3. Calculate Q-values

In [20]:
def q_values_from_values(states, actions, values, transitions, 
                         lr=1.0):

    q_values = {}
    
    for s in states:
        
        sum_values = 0
        for a in actions:
            next_state = transitions[(s, a)]
            # Q-value is value of next state + reward (?)
            v_next = values[next_state[0]] + next_state[1]
            q_values[(s, a)] = v_next
    
    return q_values

In [21]:
q_values = q_values_from_values(states, actions, values, transitions)
q_values

{((0, 0), 'l'): 0.0,
 ((0, 0), 'u'): 0.0,
 ((0, 0), 'r'): 0.0,
 ((0, 0), 'd'): 0.0,
 ((0, 1), 'l'): -1.0,
 ((0, 1), 'u'): -7.137969970703125,
 ((0, 1), 'r'): -9.35235595703125,
 ((0, 1), 'd'): -8.737396240234375,
 ((0, 2), 'l'): -7.137969970703125,
 ((0, 2), 'u'): -9.35235595703125,
 ((0, 2), 'r'): -9.967315673828125,
 ((0, 2), 'd'): -9.427825927734375,
 ((0, 3), 'l'): -9.35235595703125,
 ((0, 3), 'u'): -9.967315673828125,
 ((0, 3), 'r'): -9.967315673828125,
 ((0, 3), 'd'): -9.35235595703125,
 ((1, 0), 'l'): -7.137969970703125,
 ((1, 0), 'u'): -1.0,
 ((1, 0), 'r'): -8.737396240234375,
 ((1, 0), 'd'): -9.35235595703125,
 ((1, 1), 'l'): -7.137969970703125,
 ((1, 1), 'u'): -7.137969970703125,
 ((1, 1), 'r'): -9.427825927734375,
 ((1, 1), 'd'): -9.427825927734375,
 ((1, 2), 'l'): -8.737396240234375,
 ((1, 2), 'u'): -9.35235595703125,
 ((1, 2), 'r'): -9.35235595703125,
 ((1, 2), 'd'): -8.737396240234375,
 ((1, 3), 'l'): -9.427825927734375,
 ((1, 3), 'u'): -9.967315673828125,
 ((1, 3), 'r'):

$q_\pi(11,down)$:

In [22]:
q_values[((2, 3), 'd')]

-1.0

$q_\pi(7,down)$:

In [23]:
q_values[((1, 3), 'd')]

-7.137969970703125

In [25]:
# Is this correct?  Or should it be the same as v_next (-6.13797)?

In [24]:
values[(2, 3)]

-6.137969970703125